Here is an example notebook: all simulations were performed with the exact same scripts but different parameters to vary..

- number of observations: nr
- noise in data, or r the probability of group mis-labelling: prandom
- number of bootstraps: times (bootstrap size: p = 0.5)

The seed determining data generation was changed for each simulation. Results are processed in other notebooks. 

# Preambule

## libraries

In [1]:
library(dplyr)
library(parallel)
library(caret)
library(data.table)
library(stringr)
library("inTrees")
library(endoR)
library(qs)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: lattice

Loading required package: ggplot2


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Warning message:
“replacing previous import ‘data.table::last’ by ‘dplyr::last’ when loading ‘endoR’”
Warning message:
“replacing previous import ‘data.table::first’ by ‘dplyr::first’ when loading ‘endoR’”
Warning message:
“replacing previous import ‘data.table::between’ by ‘dplyr::between’ when loading ‘endoR’”
Warning message:
“replacing previous import ‘dplyr::union’ by ‘igraph::union’ when loading ‘endoR’”
Warning message:
“replacing previous import ‘dplyr::as_data_frame’ by ‘igraph::as_data_frame’ when loading ‘endoR’”
Warning message:
“replacing previous import ‘dplyr::groups’ by ‘igraph::groups’ w

In [2]:
library(clustermq)
options(clustermq.scheduler = "sge", clustermq.template = "~/.clustermq.tmpl")

In [3]:
sessionInfo()

R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.6 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Methanogen_SCFA/Metagenomes_methanogen/envs/r-ml/lib/libopenblasp-r0.3.10.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] clustermq_0.8.95.1 qs_0.23.4          endoR_0.1.0        inTrees_1.3       
 [5] stringr_1.4.0      data.table_1.13.4  caret_6.0-86       ggplot2_3.3.2     
 [9] lattice_0.20-41    dplyr_1.0.2       

loaded via a namespace (and not attached):
 [1] vir

## Functions

In [4]:
file.sources <- list.files('./scripts/', pattern = '*.R', full.names=TRUE, ignore.case=TRUE)
for (f in file.sources) {
    source(f)
}
rm('file.sources', 'f')

# Run

In [5]:
set.seed(0)
v_tosamp <- rnorm(n = 1e6, mean = 0.5, sd = 1)

In [6]:
tmp <- getSimulations(nr = 100, v_tosamp=v_tosamp)
tmp <- discretizeData(tmp$x, K = 2, return_split = FALSE) 
notNum <- which(!sapply(tmp, is.numeric))
for (j in notNum) set(tmp,i = NULL ,j, paste0('__', tmp[[j]]))
data_ctg <- dummyVars( ~ ., data = tmp)
data_ctg <- as.data.table(predict(data_ctg, newdata = tmp))
for (j in notNum) set(data_ctg,i = NULL ,j, str_replace(data_ctg[[j]], pattern = '^__', replacement = ''))
colnames(data_ctg) <- gsub(x = colnames(data_ctg), pattern = "`", replacement="")
colN <- colnames(data_ctg)

In [7]:
n_cores <- 20
tmpl <- list(conda = "r-ml", cores = n_cores, job_time = '24:00:00', job_mem = '15G')

In [18]:
simu100 <- Q(simWrapper, fail_on_error = FALSE
  , seedOri = 1833:1932
  , const=list(v_tosamp = v_tosamp
               , nr = 1000, prandom = 0.05
               , times = 100, p = .5
               , discretize = TRUE, K = 2, prune = TRUE
               , in_parallel = TRUE, n_cores = n_cores
               , path = '/ebio/abt3_projects/temp_data/aruaud/Toy_p005_N1000_B100/'
               )
  , export = list('metricsNet'=metricsNet, 'getSimulations'=getSimulations, 'getNetMeasures'=getNetMeasures)
  , n_jobs= 100
  , pkgs=c('data.table', 'randomForest', 'parallel', 'caret', 'stringr', 'scales', 'dplyr', 'inTrees', 'endoR', 'qs')
  , log_worker=FALSE
  , template = tmpl
 )

Warning message in (function (...) :
“Common data is 7.7 Mb. Recommended limit is  (set by clustermq.data.warning option)”
Submitting 100 worker jobs (ID: cmq7784) ...

Running 100 calculations (14 objs/7.7 Mb common; 1 calls/chunk) ...


[--------------------------------------------------]   0% (1/100 wrk) eta:  ?s

[--------------------------------------------------]   0% (2/100 wrk) eta:  ?s

[--------------------------------------------------]   0% (3/100 wrk) eta:  ?s

[--------------------------------------------------]   0% (4/100 wrk) eta:  ?s

[--------------------------------------------------]   0% (5/100 wrk) eta:  ?s

[--------------------------------------------------]   0% (6/100 wrk) eta:  ?s

[--------------------------------------------------]   0% (7/100 wrk) eta:  ?s

[--------------------------------------------------]   0% (8/100 wrk) eta:  ?s

[--------------------------------------------------]   0% (9/100 wrk) eta:  ?s

[----------------------------------------


[======================>--------------------------]  46% (55/100 wrk) eta:  9h

[======================>---------------------------]  46% (55/99 wrk) eta:  9h

[=======================>--------------------------]  47% (54/98 wrk) eta:  9h

[=======================>--------------------------]  47% (54/97 wrk) eta:  9h

[=======================>--------------------------]  47% (54/96 wrk) eta:  9h

[=======================>--------------------------]  47% (54/95 wrk) eta:  9h

[=======================>--------------------------]  47% (55/95 wrk) eta:  9h

[=======================>--------------------------]  47% (54/94 wrk) eta:  9h

[=======================>--------------------------]  47% (54/93 wrk) eta:  9h

[=======================>--------------------------]  47% (54/92 wrk) eta:  9h

[=======================>--------------------------]  47% (54/91 wrk) eta:  9h

[=======================>--------------------------]  48% (54/91 wrk) eta:  9h

[=======================>--------------


[===============================================>----]  93% (8/8 wrk) eta: 45m

[================================================>---]  94% (7/7 wrk) eta: 38m

[================================================>---]  95% (6/6 wrk) eta: 31m

[=================================================>--]  96% (5/5 wrk) eta: 25m

[=================================================>--]  97% (4/4 wrk) eta: 19m

[==================================================>-]  98% (3/3 wrk) eta: 13m

[==================================================>-]  99% (2/2 wrk) eta:  6m

[====================================================] 100% (1/1 wrk) eta:  0s
                                                                              

Master: [38023.4s 0.0% CPU]; Worker: [avg 3.4% CPU, max 840.0 Mb]

